<a href="https://colab.research.google.com/github/YuYeJi-Project/Petom/blob/teraglass/%08/temp/teraglass/petom_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 구글 드라이브 마운트

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


# 구글 드라이브에서 train, validation 압축파일 content 폴더로 카피 

In [4]:
cp /gdrive/MyDrive/Petom/train.zip ./


In [5]:
cp /gdrive/MyDrive/Petom/validation.zip ./

# train, validation 각각 폴더명으로 사용해 압축 풀기

In [ ]:
!unzip train.zip -d /content/train
!unzip validation.zip -d /content/validation

# cat, dog 폴더분리해서 train, validation 폴더 바로 밑에 오도록 path 변경
- 현미경사진(micro) : train_micro, validation_micro을 합쳐 micro.zip으로 따로 압축

In [ ]:
!tar -cvf train.zip /content/train

In [ ]:
!tar -cvf validation.zip /content/validation

In [ ]:
!tar -cvf micro.zip /content/micro

In [ ]:
import cv2
import matplotlib.pyplot as plt
import glob


In [31]:
!cd micro
!ls

micro  sample_data  train  train.zip  validation  validation.zip


In [30]:
!ls 

micro  sample_data  train  train.zip  validation  validation.zip


In [215]:
%cd /content/validation/cat/
%pwd

/content/validation/cat


'/content/validation/cat'

In [199]:
%cp *.json /content/validation/cat/json_files/

In [214]:
!zip json_files json_files

	zip warning: missing end signature--probably not a zip file (did you
	zip warning: remember to use binary mode when you transferred it?)
	zip warning: (if you are trying to read a damaged archive try -F)

zip error: Zip file structure invalid (json_files.zip)


In [ ]:
!tar -cvf json_files.zip /content/validation/cat/json_files

# json 파일 읽기

In [ ]:
# json 파일 프린트
from glob import glob
for filename in glob('*.json'):
  print(filename)

In [ ]:
# 각 파일의 shape, head 출력
from IPython.display import display
import pandas as pd
import json

data = json.load(open('IMG_C_A2_202980.json'))

# df = pd.read_json('IMG_C_A2_202980.json')
# print(filename, df.shape)
# display(df.head())
data    

In [55]:
metaData = data['metaData']
metaData

{'Path': '유증상',
 'Raw data ID': 'IMG_C_A2_18_202980.jpg',
 'age': '10',
 'breed': '코리안숏헤어',
 'camera type': 'IMG',
 'copyrighter': '㈜미소정보기술',
 'date': '2021-12-15',
 'diagnosis': '',
 'fileformat': 'jpg',
 'gender': 'M',
 'identifier': '피부질환',
 'label_path': '/라벨링데이터/반려묘/피부/일반카메라/유증상/A2_비듬_각질_상피성잔고리',
 'lesions': 'A2',
 'region': 'B',
 'resolution': '1920X1080',
 'species': 'C',
 'src_path': '/라벨링데이터/반려묘/피부/일반카메라/유증상/A2_비듬_각질_상피성잔고리',
 'type': 'json'}

In [ ]:
metaData["Raw data ID"]
metaData['breed']
metaData['age']
metaData['gender']
metaData['region']

# 필요한 칼럼 추출해서 DataFrame -> csv

# validation 
- meta 정보 섞음

In [185]:
from collections import defaultdict

meta_label = defaultdict(list)
for filename in glob('*.json'):
    data = json.load(open(filename))
    meta_label["raw_data_ID"].append(filename[:len(filename)-5])
    meta_label["species"].append(data['metaData']["species"])
    meta_label["lesions"].append(data['metaData']["lesions"])
    meta_label["breed"].append(data['metaData']["breed"])
    meta_label["age"].append(data['metaData']["age"])
    meta_label["gender"].append(data['metaData']["gender"])
    meta_label["region"].append(data['metaData']["region"])


meta_label_dt = pd.DataFrame(meta_label)
meta_label_dt.to_csv('/gdrive/MyDrive/Petom/val_cat_A2.csv')

In [184]:
meta_label_dt

,raw_data_ID,species,lesions,breed,age,gender,region
0,IMG_C_A2_202872,C,A2,코리안숏헤어,10,M,H
1,IMG_C_A2_202871,C,A2,코리안숏헤어,1,F,H
2,IMG_C_A2_202797,C,A2,페르시안,11,M,L
3,IMG_C_A2_202891,C,A2,페르시안,11,M,L
4,IMG_C_A2_202898,C,A2,러시안블루,1,M,L
...,...,...,...,...,...,...,...
333,IMG_C_A2_202762,C,A2,코리안숏헤어,4,F,L
334,IMG_C_A2_203040,C,A2,코리안숏헤어,10,F,L
335,IMG_C_A2_202957,C,A2,코리안숏헤어,11,F,L
336,IMG_C_A2_202985,C,A2,코리안숏헤어,1,F,L


In [259]:
from collections import defaultdict

label = defaultdict(list)
for filename in glob('*.json'):
    data = json.load(open(filename))
    meta_label["raw_data_ID"].append(filename[:len(filename)-5])
    meta_label["species"].append(data['metaData']["species"])
    meta_label["lesions"].append(data['metaData']["lesions"])
    meta_label["breed"].append(data['metaData']["breed"])
    meta_label["age"].append(data['metaData']["age"])
    meta_label["gender"].append(data['metaData']["gender"])
    meta_label["region"].append(data['metaData']["region"])

    ########################
    for i in range(len(data['labelingInfo'])):
        key = list(data['labelingInfo'][i].keys())[0] # 'polygon'
        polybox = data['labelingInfo'][i][key]['location'][0] # 'polygon' location
        for pb in list(polybox.keys()):
            meta_label[key+str(i%2+1)+"_"+pb].append(polybox[pb])

label

defaultdict(list, {})

In [ ]:
from collections import defaultdict

label = defaultdict(list)
for filename in glob('*.json'):
    data = json.load(open(filename))
    label['file'].append()

poly1,2 box1,2 길이 맞춰야됨

In [ ]:
meta_label_dt = pd.DataFrame(meta_label)
# meta_label_dt.to_csv('/gdrive/MyDrive/Petom/val_cat_A2.csv')
meta_label_dt

In [149]:
data['labelingInfo'][0]['polygon']['location'][0]

{'x1': 563,
 'x2': 589,
 'x3': 652,
 'x4': 720,
 'x5': 735,
 'x6': 653,
 'x7': 563,
 'y1': 557,
 'y2': 614,
 'y3': 625,
 'y4': 589,
 'y5': 522,
 'y6': 492,
 'y7': 557}

In [ ]:
from collections import defaultdict

label = defaultdict(list)
for filename in glob('*.json'):
    data = json.load(open(filename))
    ########################
    for i in range(len(data['labelingInfo'])):
        key = list(data['labelingInfo'][i].keys())[0] # 'polygon'
        polybox = data['labelingInfo'][i][key]['location'][0] # 'polygon' location
        # for pb in list(polybox.keys()):
        #     meta_label[key+str(i%2+1)+"_"+pb].append(polybox[pb])
        print(key)

In [ ]:
# 각 파일의 shape, head 출력
from IPython.display import display
import pandas as pd
import json

data = json.load(open('IMG_C_A2_202980.json'))
data

# polygon, box 2개 이상인 것들 파일 개수
- train/cat/A2 : 60
- train/dog/A1 : 16
- train/dog/A2 : 113
- train/dog/A3 : 10
- train/dog/A4 : 24
- train/dog/A5 : 81
- train/dog/A6 : 1
- validation/cat/A2 : 6
- validation/dog/A5 : 0
- validation/dog/A6 : 0




In [294]:
l = []

for filename in glob('/content/validation/dog/A6/*.json'):
    data = json.load(open(filename))   
    labelingInfo = data['labelingInfo']
    if len(labelingInfo) > 2:
        l.append(filename)
len(l)

0

In [248]:
key = labelingInfo[3].keys()
list(key)

['box']

In [267]:
%cd /content/train/cat/A2

/content/train/cat/A2


In [283]:
data = json.load(open('IMG_C_A2_200060.json'))
for i in data['labelingInfo']:
    print(i)
len(data['labelingInfo'])

{'polygon': {'color': '#1beaac', 'location': [{'x1': 1418, 'y1': 365, 'x2': 1350, 'y2': 271, 'x3': 1259, 'y3': 216, 'x4': 1171, 'y4': 215, 'x5': 1126, 'y5': 287, 'x6': 1133, 'y6': 380, 'x7': 1203, 'y7': 450, 'x8': 1298, 'y8': 454, 'x9': 1386, 'y9': 423, 'x10': 1418, 'y10': 365}], 'label': 'A2_비듬_각질_상피성잔고리', 'type': 'polygon'}}
{'polygon': {'color': '#1beaac', 'location': [{'x1': 205, 'y1': 145, 'x2': 339, 'y2': 148, 'x3': 615, 'y3': 171, 'x4': 763, 'y4': 196, 'x5': 812, 'y5': 247, 'x6': 811, 'y6': 357, 'x7': 799, 'y7': 387, 'x8': 663, 'y8': 363, 'x9': 527, 'y9': 348, 'x10': 441, 'y10': 381, 'x11': 297, 'y11': 369, 'x12': 168, 'y12': 324, 'x13': 130, 'y13': 259, 'x14': 205, 'y14': 145}], 'label': 'A2_비듬_각질_상피성잔고리', 'type': 'polygon'}}
{'box': {'color': '#1beaac', 'location': [{'x': 1126, 'y': 215, 'width': 292, 'height': 239}], 'label': 'A2_비듬_각질_상피성잔고리', 'type': 'box'}}
{'box': {'color': '#1beaac', 'location': [{'x': 130, 'y': 145, 'width': 682, 'height': 242}], 'label': 'A2_비듬_각질_상피성잔

4

In [266]:
data = json.load(open('IMG_C_A2_202789.json'))
for i in data['labelingInfo']:
    print(i)
df = pd.DataFrame()

{'polygon': {'color': '#1beaac', 'location': [{'x1': 457, 'y1': 622, 'x2': 344, 'y2': 622, 'x3': 196, 'y3': 539, 'x4': 190, 'y4': 473, 'x5': 261, 'y5': 368, 'x6': 363, 'y6': 323, 'x7': 405, 'y7': 330, 'x8': 486, 'y8': 497, 'x9': 457, 'y9': 622}], 'label': 'A2_비듬_각질_상피성잔고리', 'type': 'polygon'}}
{'polygon': {'color': '#1beaac', 'location': [{'x1': 1124, 'y1': 297, 'x2': 1190, 'y2': 273, 'x3': 1311, 'y3': 337, 'x4': 1330, 'y4': 424, 'x5': 1276, 'y5': 492, 'x6': 1176, 'y6': 478, 'x7': 1152, 'y7': 421, 'x8': 1124, 'y8': 297}], 'label': 'A2_비듬_각질_상피성잔고리', 'type': 'polygon'}}
{'box': {'color': '#1beaac', 'location': [{'x': 190, 'y': 323, 'width': 296, 'height': 299}], 'label': 'A2_비듬_각질_상피성잔고리', 'type': 'box'}}
{'box': {'color': '#1beaac', 'location': [{'x': 1124, 'y': 273, 'width': 206, 'height': 219}], 'label': 'A2_비듬_각질_상피성잔고리', 'type': 'box'}}


In [ ]:
df = pd.DataFrame()
for i in data['labelingInfo']:
    for k in i:

        df.append(i,ignore_index=True)
df

In [ ]:
%cd

In [ ]:
for filename in glob('*.json'):
    data = json.load(open(filename))   
    try:
        box = data['labelingInfo'][1]['box']['location'][0]
    except:
        print(filename)